In [255]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from pydantic import BaseModel
import importlib
import shutil
from utils import utils
import pandas as pd
from utils.time_converter import TimeConverter
importlib.reload(utils)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [256]:
subject = '수학'
room_id = '674e58163925f28f6caf4fa0'

file_keys = utils.get_items('pagecall-text', f'{subject}/{room_id}')
utils.download_items('pagecall-text', file_keys, './downloads')

In [257]:
raw_data = utils.merge_files('./downloads')
shutil.rmtree('./downloads')

In [258]:
teacher_extracted_data = utils.extract_speaker(raw_data, speaker='teacher')
teacher_splited_data = utils.split_sentences(teacher_extracted_data)
student_extracted_data = utils.extract_speaker(raw_data, speaker='student')
student_splited_data = utils.split_sentences(student_extracted_data)

In [259]:
teacher_splited_data = utils.mapping_time(teacher_extracted_data, teacher_splited_data)
teacher_splited_data

[{'idx': 0, 'text': '목소리 잘 들릴까요?', 'start': '4m 54.5s', 'end': '4m 58.3s'},
 {'idx': 1, 'text': '안녕하세요.', 'start': '4m 58.5s', 'end': '5m 0.6s'},
 {'idx': 2,
  'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.',
  'start': '5m 0.7s',
  'end': '5m 6.2s'},
 {'idx': 3, 'text': '잘 보여요?', 'start': '5m 9.7s', 'end': '5m 12.5s'},
 {'idx': 4, 'text': '잠깐만요.', 'start': '5m 12.5s', 'end': '5m 12.6s'},
 {'idx': 5,
  'text': '혹시 전에 선택으로 과외 해본 적 있어요?',
  'start': '5m 23.3s',
  'end': '5m 35.5s'},
 {'idx': 6, 'text': '아 정말요?', 'start': '5m 35.5s', 'end': '5m 38.1s'},
 {'idx': 7,
  'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?',
  'start': '5m 38.8s',
  'end': '5m 57.2s'},
 {'idx': 8,
  'text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요',
  'start': '5m 59.9s',
  'end': '6m 23.0s'},
 {'idx': 9, 'text': '아 그거 괜찮아요', 'start': '5m 59.9s', 'end': '6m 23.0s'},
 {'idx': 10,
  'text': '같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.',
  'start': '5m 59.9s',
  'end': '6m 33.3s'},
 {'idx': 11,
  'text': '그래서 이 파트가 지금 하반기용이고, 이

In [260]:
student_splited_data = utils.mapping_time(student_extracted_data, student_splited_data)
student_splited_data

[{'idx': 0, 'text': '감사합니다.', 'start': '0m 0.0s', 'end': '0m 2.4s'},
 {'idx': 1, 'text': '네 안녕하세요', 'start': '4m 41.8s', 'end': '5m 10.4s'},
 {'idx': 2, 'text': '네 네', 'start': '4m 41.8s', 'end': '5m 10.4s'},
 {'idx': 3, 'text': '네, 같이 볼게요.', 'start': '5m 35.0s', 'end': '5m 55.5s'},
 {'idx': 4,
  'text': '어떤 걸 보여줄지 모르겠어가지고..',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 5, 'text': '아니 근데..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 6,
  'text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 7,
  'text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 8, 'text': '그..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 9, 'text': '모르는 게 너무 많아가지고..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 10, 'text': '네..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 11,
  'text': '아니 진짜 충격부모일 수도 있어요.',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 12, 'text': '네..', 'start': '6m 33.2s', 'end':

In [261]:
teacher_df = pd.DataFrame(teacher_splited_data).rename(columns={"idx": "teacher_idx", "text": "teacher_text"})
student_df = pd.DataFrame(student_splited_data).rename(columns={"idx": "student_idx", "text": "student_text"})

teacher_df['start_ms'] = teacher_df['start'].apply(lambda x: TimeConverter.convert_time_to_milliseconds(x))
student_df['start_ms'] = student_df['start'].apply(lambda x: TimeConverter.convert_time_to_milliseconds(x))

df = pd.concat([teacher_df, student_df], ignore_index=True)
df = df.sort_values(by=["start_ms", "teacher_idx", "student_idx"]).reset_index(drop=True).drop(columns=["start_ms"])
df = df.astype({'teacher_idx': 'Int64', 'student_idx': 'Int64'})
df['time'] = df['start'] + " ~ " + df['end']
df = df.drop(columns=['start', 'end'])
df = df[['teacher_idx', 'student_idx', 'time', 'teacher_text', 'student_text']]

teacher_df = df[df['teacher_text'].notnull()].drop(columns=['student_text', 'student_idx', 'time']).rename(columns={"teacher_idx": "idx", "teacher_text": "text"}).reset_index(drop=True)
student_df = df[df['student_text'].notnull()].drop(columns=['teacher_text', 'teacher_idx', 'time']).rename(columns={"student_idx": "idx", "student_text": "text"}).reset_index(drop=True)
df

,teacher_idx,student_idx,time,teacher_text,student_text
0,<NA>,0,0m 0.0s ~ 0m 2.4s,NaN,감사합니다.
1,<NA>,1,4m 41.8s ~ 5m 10.4s,NaN,네 안녕하세요
2,<NA>,2,4m 41.8s ~ 5m 10.4s,NaN,네 네
3,0,<NA>,4m 54.5s ~ 4m 58.3s,목소리 잘 들릴까요?,NaN
4,1,<NA>,4m 58.5s ~ 5m 0.6s,안녕하세요.,NaN
...,...,...,...,...,...
1341,<NA>,441,117m 52.3s ~ 117m 52.6s,NaN,안녕히 계세요.
1342,900,<NA>,118m 7.7s ~ 118m 8.4s,안녕하세요.,NaN
1343,901,<NA>,118m 8.5s ~ 118m 11.0s,처음인데 미안해요.,NaN
1344,902,<NA>,118m 11.1s ~ 118m 11.9s,미안하네요.,NaN


In [262]:
teacher_df

,idx,text
0,0,목소리 잘 들릴까요?
1,1,안녕하세요.
2,2,잘 들리고 그러면 필기 한번 확인해 볼게요.
3,3,잘 보여요?
4,4,잠깐만요.
...,...,...
899,899,네 그냥 나가면 되는 걸까요?
900,900,안녕하세요.
901,901,처음인데 미안해요.
902,902,미안하네요.


In [263]:
chunks_with_overlap = utils.split_with_overlap(teacher_df, chunk_size=30, overlap=5)

In [264]:
chunks_with_overlap

[[{'idx': 0, 'text': '목소리 잘 들릴까요?'},
  {'idx': 1, 'text': '안녕하세요.'},
  {'idx': 2, 'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
  {'idx': 3, 'text': '잘 보여요?'},
  {'idx': 4, 'text': '잠깐만요.'},
  {'idx': 5, 'text': '혹시 전에 선택으로 과외 해본 적 있어요?'},
  {'idx': 6, 'text': '아 정말요?'},
  {'idx': 7, 'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?'},
  {'idx': 8, 'text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요'},
  {'idx': 9, 'text': '아 그거 괜찮아요'},
  {'idx': 10,
   'text': '같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.'},
  {'idx': 11, 'text': '그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요.'},
  {'idx': 12,
   'text': '그래서 일단은 지금 하반기니까, 하반기 거를 일단 하고, 기말고사 얼마 안 남았으니까 샌쩍 같이 본 다음에 시간이 남으면 상반기 걸 같이 볼게요'},
  {'idx': 13, 'text': '그리고 제가 이거 답을 한번 봤는데 일단 풍거는 거의 다 맞았던 것 같아요'},
  {'idx': 14,
   'text': '네 그래서 풍거는 잘 푸는 것 같고 별표 위주로 한번 보면 될 것 같고 그 다음에 이게 지금 교재가 깨끗한 상태잖아요.'},
  {'idx': 15, 'text': '근데 하윤 학생이 뭘 틀렸는지 제가 모르니까.'},
  {'idx': 16, 'text': '이거를 하윤 학생이 그때그때 말해주면 좋을 것 같아요.'},
  {'idx': 17, 'text': '이 시간이 왜 지금 가고 있는거지?'},


In [265]:
from prompt import question_analyzer

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_analyzer)
system_prompt = question_analyzer.QuestionAnalyzer(subject, user='선생님').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [266]:
# question_indices = utils.extract_question_indices(question_analysis_results)
# question_context = utils.get_question_context(teacher_df.to_dict(orient='records'), question_indices)
# question_context
question_indices = utils.extract_question_indices(question_analysis_results)
question_indices = df[df['teacher_idx'].isin(question_indices)].index.tolist()

question_context = utils.get_question_context_v2(df.to_dict(orient='records'), question_indices)
question_context

[{'idx': 3,
  'question': '목소리 잘 들릴까요?',
  'context': [{'time': '0m 0.0s ~ 0m 2.4s', 'student_text': '감사합니다.'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teacher_text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
   {'time': '5m 9.7s ~ 5m 12.5s', 'teacher_text': '잘 보여요?'},
   {'time': '5m 12.5s ~ 5m 12.6s', 'teacher_text': '잠깐만요.'},
   {'time': '5m 23.3s ~ 5m 35.5s',
    'teacher_text': '혹시 전에 선택으로 과외 해본 적 있어요?'}]},
 {'idx': 6,
  'question': '잘 보여요?',
  'context': [{'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teacher_text': '잘 들리고 

In [267]:
from prompt import question_classifier

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_classifier)
system_prompt = question_classifier.QuestionClassifier(subject, user='선생님').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in question_context
])

In [268]:
for (res,item) in zip(question_classifier_results, question_context):
    item['result'] = res
question_context

[{'idx': 3,
  'question': '목소리 잘 들릴까요?',
  'context': [{'time': '0m 0.0s ~ 0m 2.4s', 'student_text': '감사합니다.'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teacher_text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
   {'time': '5m 9.7s ~ 5m 12.5s', 'teacher_text': '잘 보여요?'},
   {'time': '5m 12.5s ~ 5m 12.6s', 'teacher_text': '잠깐만요.'},
   {'time': '5m 23.3s ~ 5m 35.5s', 'teacher_text': '혹시 전에 선택으로 과외 해본 적 있어요?'}],
  'result': 'False'},
 {'idx': 6,
  'question': '잘 보여요?',
  'context': [{'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teach

In [269]:
# feed_idx = [item['idx'] for item in question_context if item['result'] == 'True']
# feed_idx = df[df['teacher_idx'].isin(feed_idx)].index.tolist()
# final_feed = utils.get_question_context_v2(df.to_dict(orient='records'), feed_idx)
# final_feed
final_feed = [{'idx': item['idx'], 'question': item['question'], 'context': item['context']} for item in question_context if item['result'] == 'True']
final_feed

[{'idx': 127,
  'question': '으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 어떤 거부터 시작한다는 게 정확히 무슨 말이에요?',
  'context': [{'time': '14m 55.7s ~ 14m 58.2s', 'teacher_text': '네.'},
   {'time': '14m 58.9s ~ 14m 58.9s', 'teacher_text': '네.'},
   {'time': '14m 59.8s ~ 14m 59.9s', 'teacher_text': '네.'},
   {'time': '15m 2.6s ~ 15m 2.6s', 'teacher_text': '네.'},
   {'time': '15m 14.3s ~ 15m 22.3s',
    'student_text': '무엇을 알고 있는지는 모르겠고 그리고 어려운 문제 금방 보고 이거 진짜 완전 좀 심해요.'},
   {'time': '15m 20.0s ~ 15m 53.0s',
    'teacher_text': '으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 어떤 거부터 시작한다는 게 정확히 무슨 말이에요?'},
   {'time': '15m 22.5s ~ 15m 32.8s',
    'student_text': '어려운 문제 문제 조금이라도 길거나 이해 안 되면 바로 막 별표 치고 넘겨가지고 그리고 다 해당하는 것 같아요.'},
   {'time': '15m 35.0s ~ 15m 35.7s', 'student_text': '약간'},
   {'time': '15m 42.8s ~ 16m 0.6s', 'student_text': '어떤 것부터 시작해야 하는지 모르겠어요.'},
   {'time': '15m 53.0s ~ 16m 23.0s',
    'teacher_text': '네 네 네 네 네 어떻게 공부해야 될지 잘 모르겠다.'},
   {'time': '16m 0.8s ~ 16m 4.6s',
    'student_text': '중간고사 공부할 때 학원에서 딱 

In [270]:
# final_feed = [{'idx': item['idx'], 'question': item['question'], 'context': item['context']} for item in question_context if item['result'] == 'True']
# final_feed

In [271]:
from prompt import digging_v2

llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(digging_v2)
system_prompt = digging_v2.Digging(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
results = chain.batch([
    {"user_message": chunk} for chunk in final_feed
])

In [272]:
results = [json.loads(item.strip('```json').strip('```').strip()) for item in results]

for (res, item) in zip(results, final_feed):
    item['result'] = res['result']
    # item['answer'] = res['answer']
    item['reason'] = res['reason']
final_feed

[{'idx': 127,
  'question': '으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 어떤 거부터 시작한다는 게 정확히 무슨 말이에요?',
  'context': [{'time': '14m 55.7s ~ 14m 58.2s', 'teacher_text': '네.'},
   {'time': '14m 58.9s ~ 14m 58.9s', 'teacher_text': '네.'},
   {'time': '14m 59.8s ~ 14m 59.9s', 'teacher_text': '네.'},
   {'time': '15m 2.6s ~ 15m 2.6s', 'teacher_text': '네.'},
   {'time': '15m 14.3s ~ 15m 22.3s',
    'student_text': '무엇을 알고 있는지는 모르겠고 그리고 어려운 문제 금방 보고 이거 진짜 완전 좀 심해요.'},
   {'time': '15m 20.0s ~ 15m 53.0s',
    'teacher_text': '으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 어떤 거부터 시작한다는 게 정확히 무슨 말이에요?'},
   {'time': '15m 22.5s ~ 15m 32.8s',
    'student_text': '어려운 문제 문제 조금이라도 길거나 이해 안 되면 바로 막 별표 치고 넘겨가지고 그리고 다 해당하는 것 같아요.'},
   {'time': '15m 35.0s ~ 15m 35.7s', 'student_text': '약간'},
   {'time': '15m 42.8s ~ 16m 0.6s', 'student_text': '어떤 것부터 시작해야 하는지 모르겠어요.'},
   {'time': '15m 53.0s ~ 16m 23.0s',
    'teacher_text': '네 네 네 네 네 어떻게 공부해야 될지 잘 모르겠다.'},
   {'time': '16m 0.8s ~ 16m 4.6s',
    'student_text': '중간고사 공부할 때 학원에서 딱 

In [273]:
with open(f"{room_id}.txt", "w", encoding="utf-8-sig") as file:
    json.dump(final_feed, file, ensure_ascii=False, indent=4)